# Field of View Asynchronous Calculation Notebook

This Jupyter Notebook shows how to use the SatChecker FOV API to calculate the field of view of a satellite pass asynchronously using the `satellite-passes` and `task-status` endpoints.

The primary difference between using the async parameter and not using it is that the async version returns a task ID, which is then used in the `task-status` endpoint to check the status of the task and retrieve the results. This will prevent the query from timing out.

### Example URLs for browser usage:
**FOV Query:**

https://satchecker.cps.iau.org/fov/satellite-passes/?latitude=-30.2446&longitude=-70.7494&elevation=2647&mid_obs_time_jd=2460208.5&duration=3600&ra=180&dec=-30&fov_radius=2&group_by=satellite

**Result from FOV Query:** (partial)

"task_id": "bcf1680e-390e-48fa-a529-8d941677aad7"

<hr>

**Get Task Status:**

https://satchecker.cps.iau.org/fov/task-status/bcf1680e-390e-48fa-a529-8d941677aad7

**Result from Get Task Status:**
```json
{
    "data": {...}, 
    "message": "FOV calculation completed successfully",
    "performance": {...},
    "source": "IAU CPS SatChecker",
    "status": "SUCCESS",
    "task_id": "bcf1680e-390e-48fa-a529-8d941677aad7",
    "version": "1.6.0"
}
```



In [ ]:
import json
import time

import requests

BASE_URL = "https://satchecker.cps.iau.org"

In [2]:
# Step 1: Submit async FOV request
response = requests.get(
    f"{BASE_URL}/fov/satellite-passes/",
    params={
        "latitude": -30.2446,
        "longitude": -70.7494,
        "elevation": 2647.0,
        "ra": 180.0,
        "dec": -30.0,
        "fov_radius": 2.0,
        "duration": 3600.0,
        "mid_obs_time_jd": 2460208.5,
        "group_by": "satellite",
        "async": True,  # Added for clarity, this is the default
    },
    timeout=60,
)

response.raise_for_status()

task_id = response.json()["task_id"]
print(json.dumps(response.json(), indent=4))

{
    "api_source": "IAU CPS SatChecker",
    "api_version": "1.6.0",
    "message": "FOV calculation started. Use the task_id to check status and retrieve results.",
    "status": "PENDING",
    "task_id": "e6facdb4-b455-419d-ba71-4f40b01d78d2"
}


In [3]:
# Step 2: Poll for results
while True:
    status_response = requests.get(
        f"{BASE_URL}/fov/task-status/{task_id}", timeout=30
    )
    status_response.raise_for_status()
    status_data = status_response.json()

    status = status_data["status"]

    if status == "SUCCESS":
        print("Task completed!")
        data = status_data.get('data', {})
        print(f"Found {data.get('total_satellites', 0)} satellites")
        print(f"Total position results: {data.get('total_position_results', 0)}")
        
        # Uncomment to print the first satellite to see the format
        # The contents of the "data" key is the same as the response from the 
        # "satellite-passes" endpoint with async=False
        
        #satellites = data.get('satellites', {})
        #first_satellite = dict(list(satellites.items())[:1]) if satellites else {}
        
        #print("\n" + "="*50)
        #print("First Satellite (JSON formatted):")
        #print("="*50)
        #print(json.dumps(first_satellite, indent=2))

        break
    elif status == "FAILURE":
        print(f"Task failed: {status_data.get('error')}")
        break
    elif status == "PROGRESS":
        progress = status_data.get("progress", 0)
        print(f"Progress: {progress:.1f}%")

    time.sleep(2)  # Wait 2 seconds before checking again



Progress: 1.0%
Progress: 5.0%
Progress: 8.0%
Progress: 12.0%
Progress: 15.0%
Progress: 18.0%
Progress: 22.0%
Progress: 25.0%
Progress: 29.0%
Progress: 32.0%
Progress: 36.0%
Progress: 39.0%
Progress: 44.0%
Progress: 48.0%
Progress: 51.0%
Progress: 55.0%
Progress: 58.0%
Progress: 62.0%
Progress: 65.0%
Progress: 68.0%
Progress: 72.0%
Progress: 74.0%
Progress: 79.0%
Progress: 82.0%
Progress: 86.0%
Progress: 89.0%
Progress: 91.0%
Progress: 94.0%
Task completed!
Found 194 satellites
Total position results: 6148
